# IOS-XE CLEU 2020 Demo

## Connecting to a Device

Let's define some variables:

In [1]:
# Local CSR 1000v (running under vagrant) -- rtr1
HOST = '127.0.0.1'
PORT = 2223
USER = 'vagrant'
PASS = 'vagrant'

Now let's establish a NETCONF session to that box using ncclient:

In [2]:
from ncclient import manager
from lxml import etree

def pretty_print(retval):
        print(etree.tostring(retval.data, pretty_print=True).decode())

def my_unknown_host_cb(host, fingerprint):
    return True
    
m = manager.connect(host=HOST, port=PORT, username=USER, password=PASS,
                    allow_agent=False,
                    look_for_keys=False,
                    hostkey_verify=False,
                    unknown_host_cb=my_unknown_host_cb)

In [3]:
m

## Capabilities

Let's look at the capabilities presented by the thing we've just connected to:

In [4]:
for c in m.server_capabilities:
    print(c)

urn:ietf:params:netconf:base:1.0
urn:ietf:params:netconf:base:1.1
urn:ietf:params:netconf:capability:writable-running:1.0
urn:ietf:params:netconf:capability:xpath:1.0
urn:ietf:params:netconf:capability:validate:1.0
urn:ietf:params:netconf:capability:validate:1.1
urn:ietf:params:netconf:capability:rollback-on-error:1.0
urn:ietf:params:netconf:capability:notification:1.0
urn:ietf:params:netconf:capability:interleave:1.0
urn:ietf:params:netconf:capability:with-defaults:1.0?basic-mode=explicit&also-supported=report-all-tagged
urn:ietf:params:netconf:capability:yang-library:1.0?revision=2016-06-21&module-set-id=61de1a2313e60afe62df413a77ed9087
http://tail-f.com/ns/netconf/actions/1.0
http://tail-f.com/ns/netconf/extensions
http://cisco.com/ns/cisco-xe-ietf-ip-deviation?module=cisco-xe-ietf-ip-deviation&revision=2016-08-10
http://cisco.com/ns/cisco-xe-ietf-ipv4-unicast-routing-deviation?module=cisco-xe-ietf-ipv4-unicast-routing-deviation&revision=2015-09-11
http://cisco.com/ns/cisco-xe-ietf-

Ok, that's a bit messy, so let's tidy it up a bit and look, initially, at all the base netconf capabilities:

In [5]:
nc_caps = [c for c in m.server_capabilities if c.startswith('urn:ietf:params:netconf') and "smiv2" not in c]
for c in nc_caps:
    print(c)

urn:ietf:params:netconf:base:1.0
urn:ietf:params:netconf:base:1.1
urn:ietf:params:netconf:capability:writable-running:1.0
urn:ietf:params:netconf:capability:xpath:1.0
urn:ietf:params:netconf:capability:validate:1.0
urn:ietf:params:netconf:capability:validate:1.1
urn:ietf:params:netconf:capability:rollback-on-error:1.0
urn:ietf:params:netconf:capability:notification:1.0
urn:ietf:params:netconf:capability:interleave:1.0
urn:ietf:params:netconf:capability:with-defaults:1.0?basic-mode=explicit&also-supported=report-all-tagged
urn:ietf:params:netconf:capability:yang-library:1.0?revision=2016-06-21&module-set-id=61de1a2313e60afe62df413a77ed9087


And now let's look at the capabilities that are related to model support:

In [ ]:
import re

for c in m.server_capabilities:
    model = re.search('module=([^&]*)&', c)
    if model is not None:
        print("{}".format(model.group(1)))

        revision = re.search('revision=([0-9]{4}-[0-9]{2}-[0-9]{2})', c)
        if revision is not None:
            print("  revision = {}".format(revision.group(1)))

        deviations = re.search('deviations=([a-zA-Z0-9\-,]+)($|&)',c)
        if deviations is not None:
            print("  deviations = {}".format(deviations.group(1)))

        features = re.search('features=([a-zA-Z0-9\-,]+)($|&)',c)
        if features is not None:
            print("  features = {}".format(features.group(1)))

## Schema

Let's take a look at playing with schema. First, we can try downloading them, picking one of the modules we got capabilities for.

In [6]:
# SCHEMA_TO_GET = 'openconfig-interfaces'
# SCHEMA_TO_GET = 'Cisco-IOS-XE-native'
# SCHEMA_TO_GET = 'Cisco-IOS-XE-features'
SCHEMA_TO_GET = 'Cisco-IOS-XE-interfaces-oper'

c = m.get_schema(SCHEMA_TO_GET)
print(c.data)

module Cisco-IOS-XE-interfaces-oper {
  yang-version 1;
  namespace "http://cisco.com/ns/yang/Cisco-IOS-XE-interfaces-oper";
  prefix interfaces-ios-xe-oper;

  import ietf-inet-types {
    prefix inet;
  }
  import ietf-yang-types {
    prefix yang;
  }
  import cisco-semver {
    prefix cisco-semver;
  }

  organization
    "Cisco Systems, Inc.";
  contact
    "Cisco Systems, Inc.
     Customer Service
     
     Postal: 170 W Tasman Drive
     San Jose, CA 95134
     
     Tel: +1 1800 553-NETS
     
     E-mail: cs-yang@cisco.com";
  description
    "This module contains a collection of YANG definitions for
     monitoring the interfaces in a Network Element.
     Copyright (c) 2016-2019 by Cisco Systems, Inc.
     All rights reserved.";

  revision 2019-11-01 {
    description
      "- Added media type to interface
       - Added more interface speed values";
    reference "3.3.0";
    cisco-semver:module-version "3.3.0";
  }
  revision 2019-05-01 {
    description
      "- Added 

That's not so readable. Let's use a utility called ```pyang``` to get something a bit more readable.

In [ ]:
from subprocess import Popen, PIPE, STDOUT

SCHEMA_TO_GET = 'Cisco-IOS-XE-interfaces-oper'

c = m.get_schema(SCHEMA_TO_GET)

# Simple pyang tree display
# p = Popen(['pyang', '-f', 'tree'], stdout=PIPE, stdin=PIPE, stderr=PIPE)

# Restrict display depth
# p = Popen(['pyang', '-f', 'tree', '--tree-depth', '2'], stdout=PIPE, stdin=PIPE, stderr=PIPE)

# Restrict display path
p = Popen(['pyang',
           '-f', 'tree',
           '--tree-path', 'interfaces/interface/statistics'],
          stdout=PIPE, stdin=PIPE, stderr=PIPE)

# Push the data from the get_schema through pyang
stdout_data = p.communicate(input=c.data.encode())[0]

print(stdout_data.decode())

## What About Config?

The ncclient library provides for some simple operations. Let's skip thinking about schemas and stuff like that. Instead let's focus on config and getting end setting it. For that, ncclient provides two methods:

* get_config - takes a target data store and an optional filter
* edit_config - takes a target data store and an XML document with the edit request

### Getting Config

Let's look at some simple requests...

In [ ]:
c = m.get_config(source='running')
pretty_print(c)

Now let's add in a simple filter:

In [7]:
filter = '''
<native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
 <username/>
</native>
'''

with m.locked(target='running'):
    c = m.get_config(source='running', filter=('subtree', filter))
    pretty_print(c)

<data xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
    <username>
      <name>vagrant</name>
      <privilege>15</privilege>
      <password>
        <encryption>0</encryption>
        <password>vagrant</password>
      </password>
    </username>
  </native>
</data>



### Retrieve Interface Config Data (Native Model)

In [8]:
filter = '''
<native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
  <interface/>
</native>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

<data xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
    <interface>
      <GigabitEthernet>
        <name>1</name>
        <ip>
          <address>
            <dhcp/>
          </address>
          <igmp xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-igmp">
            <explicit-tracking>false</explicit-tracking>
            <proxy-service>false</proxy-service>
            <unidirectional-link>false</unidirectional-link>
            <v3lite>false</v3lite>
          </igmp>
        </ip>
        <mop>
          <enabled>false</enabled>
          <sysid>false</sysid>
        </mop>
        <speed xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-ethernet">
          <value-1000/>
        </speed>
        <negotiation xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-ethernet">
          <auto>false</auto>
        </negotiation>
      </GigabitEthernet>
      <GigabitEthernet>
      

### Retrieve Interface Data (Native Model) With XPath Query

As well as subtree filters, **IOS-XE** supports XPath-based filters.

In [9]:
filter = '/native/interface/GigabitEthernet/name'

c = m.get_config(source='running', filter=('xpath', filter))
pretty_print(c)

<data xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
    <interface>
      <GigabitEthernet>
        <name>1</name>
      </GigabitEthernet>
      <GigabitEthernet>
        <name>2</name>
      </GigabitEthernet>
      <GigabitEthernet>
        <name>3</name>
      </GigabitEthernet>
      <GigabitEthernet>
        <name>4</name>
      </GigabitEthernet>
    </interface>
  </native>
</data>



### Retrieve All BGP Data

Now let's look at the BGP native model:

In [ ]:
filter = '''
<native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
 <router>
   <bgp xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-bgp"/>
 </router>
</native>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

### Look At A Specific BGP Neighbor

And can we look at a specific neighbor only? Say the one with id (address) ```2.2.2.3```?

In [ ]:
filter = '''
<native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
  <router>
    <bgp xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-bgp">
      <id>100</id>
      <neighbor>
        <id>2.2.2.3</id>
      </neighbor>
        <address-family>
          <no-vrf>
            <ipv4>
              <af-name>unicast</af-name>
              <ipv4-unicast>
                <neighbor>
                  <id>2.2.2.3</id>
                </neighbor>
              </ipv4-unicast>
            </ipv4>
          </no-vrf>
        </address-family>
    </bgp>
  </router>
</native>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

### Create New BGP Neighbor

Ok, so, yes we can get a specific neighbor. Now, can we create a new neighbor? Let's create one with an id of `2.2.2.4`, with a remote-as of 666.

In [10]:
from ncclient.operations import TimeoutExpiredError

edit_data = '''
<config xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
    <router>
      <bgp xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-bgp">
        <id>100</id>
        <neighbor>
          <id>2.2.2.4</id>
          <remote-as>666</remote-as>
        </neighbor>
        <address-family>
          <no-vrf>
            <ipv4>
              <af-name>unicast</af-name>
              <ipv4-unicast>
                <neighbor>
                  <id>2.2.2.4</id>
                  <activate/>
                </neighbor>
              </ipv4-unicast>
            </ipv4>
          </no-vrf>
        </address-family>
      </bgp>
    </router>
  </native>
</config>
'''

try:
    with m.locked(target='running'):
        edit_reply = m.edit_config(edit_data, target='running', format='xml')
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

Now let's pull back some of the data for that neighbor:

In [11]:
filter = '''
<native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
  <router>
    <bgp xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-bgp">
      <id>100</id>
      <neighbor>
        <id>2.2.2.4</id>
      </neighbor>
    </bgp>
  </router>
</native>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

<data xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
    <router>
      <bgp xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-bgp">
        <id xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">100</id>
        <neighbor>
          <id xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">2.2.2.4</id>
          <remote-as>666</remote-as>
          <timers>
            <keepalive-interval>60</keepalive-interval>
            <holdtime>180</holdtime>
          </timers>
        </neighbor>
      </bgp>
    </router>
  </native>
</data>



### Modify The BGP Neighbor Description

Can modify something in the neighbor we just created? Let's keep it simple and modify the description:

In [ ]:
from ncclient.operations import TimeoutExpiredError

edit_data = '''
<config xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
    <router>
      <bgp xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-bgp">
        <id>100</id>
        <neighbor>
          <id>2.2.2.4</id>
          <description nc:operation="merge">*** MODIFIED DESCRIPTION ***</description>
        </neighbor>
      </bgp>
    </router>
  </native>
</config>
'''

try:
    with m.locked(target='running'):
        edit_reply = m.edit_config(edit_data, target='running', format='xml')
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

### Delete A BGP Neighbor

In [13]:
from ncclient.operations import TimeoutExpiredError
from lxml.etree import XMLSyntaxError

edit_data = '''
<config xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
    <router>
      <bgp xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-bgp">
        <id>100</id>
        <neighbor nc:operation="delete">
          <id>2.2.2.4</id>
        </neighbor>
        <address-family>
          <no-vrf>
            <ipv4>
              <af-name>unicast</af-name>
              <ipv4-unicast>
                <neighbor nc:operation="delete">
                  <id>2.2.2.4</id>
                </neighbor>
              </ipv4-unicast>
            </ipv4>
          </no-vrf>
        </address-family>
      </bgp>
    </router>
  </native>
</config>
'''

try:
    with m.locked(target='running'):
        edit_reply = m.edit_config(edit_data, target='running', format='xml')
except TimeoutExpiredError as e:
    print("Operation timeout!")
except XMLSyntaxError as e:
    print(e)
    print(e.args)
    print(dir(e))
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

severity=error, tag=data-missing
{'type': 'application', 'tag': 'data-missing', 'severity': 'error', 'info': '<?xml version="1.0" encoding="UTF-8"?><error-info xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0"><bad-element>neighbor</bad-element>\n</error-info>\n', 'path': "\n    /nc:rpc/nc:edit-config/nc:config/ios:native/ios:router/ios-bgp:bgp[ios-bgp:id='100']/ios-bgp:neighbor[ios-bgp:id='2.2.2.4']\n  ", 'message': None}


## Other Stuff

### Get interface operational data from native model:

In [14]:
filter = '''
<interfaces xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-interfaces-oper">
  <interface/>
</interfaces>
'''
c = m.get(filter=('subtree', filter))
pretty_print(c)

<data xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <interfaces xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-interfaces-oper">
    <interface>
      <name>Control Plane</name>
      <interface-type>iana-iftype-other</interface-type>
      <admin-status>if-state-up</admin-status>
      <oper-status>if-oper-state-ready</oper-status>
      <last-change>2020-01-19T15:11:26.873+00:00</last-change>
      <if-index>0</if-index>
      <phys-address>00:00:00:00:00:00</phys-address>
      <speed>10240000000</speed>
      <statistics>
        <discontinuity-time>2020-01-19T15:09:37+00:00</discontinuity-time>
        <in-octets>0</in-octets>
        <in-unicast-pkts>0</in-unicast-pkts>
        <in-broadcast-pkts>0</in-broadcast-pkts>
        <in-multicast-pkts>0</in-multicast-pkts>
        <in-discards>0</in-discards>
        <in-errors>0</in-errors>
        <in-unknown-protos>0</in-unknown-protos>
        <out-octets>0</out-octets>
        <

### Get all interface names

In [15]:
filter = '''
<interfaces xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-interfaces-oper">
  <interface>
    <name/>
  </interface>
</interfaces>
'''
c = m.get(filter=('subtree', filter))
pretty_print(c)

<data xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <interfaces xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-interfaces-oper">
    <interface>
      <name>Control Plane</name>
    </interface>
    <interface>
      <name>GigabitEthernet1</name>
    </interface>
    <interface>
      <name>GigabitEthernet2</name>
    </interface>
    <interface>
      <name>GigabitEthernet3</name>
    </interface>
    <interface>
      <name>GigabitEthernet4</name>
    </interface>
    <interface>
      <name>Loopback100</name>
    </interface>
  </interfaces>
</data>



### Get the statistics from a specific interface:

In [16]:
filter = '''
<interfaces xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-interfaces-oper">
  <interface>
    <name>GigabitEthernet1</name>
    <statistics/>
  </interface>
</interfaces>
'''
c = m.get(filter=('subtree', filter))
pretty_print(c)

<data xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <interfaces xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-interfaces-oper">
    <interface>
      <name xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">GigabitEthernet1</name>
      <statistics>
        <discontinuity-time>2020-01-19T15:09:37+00:00</discontinuity-time>
        <in-octets>797761</in-octets>
        <in-unicast-pkts>7650</in-unicast-pkts>
        <in-broadcast-pkts>0</in-broadcast-pkts>
        <in-multicast-pkts>0</in-multicast-pkts>
        <in-discards>0</in-discards>
        <in-errors>0</in-errors>
        <in-unknown-protos>0</in-unknown-protos>
        <out-octets>6697678</out-octets>
        <out-unicast-pkts>6734</out-unicast-pkts>
        <out-broadcast-pkts>0</out-broadcast-pkts>
        <out-multicast-pkts>0</out-multicast-pkts>
        <out-discards>0</out-discards>
        <out-errors>0</out-errors>
        <rx-pps>0</rx-pps>
        <rx-kbps>0</rx-

### Exercise Constraints

#### Create a VRF and put an interface into it

In [ ]:
from ncclient.operations import TimeoutExpiredError

edit_data = '''
<config xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
    <interface>
      <GigabitEthernet>
        <name>4</name>
        <vrf>
          <forwarding>TEST</forwarding>
        </vrf>
        <ip>
          <address>
            <primary>
              <address>192.168.1.222</address>
              <mask>255.255.255.0</mask>
            </primary>
          </address>
        </ip>
      </GigabitEthernet>
    </interface>
    <vrf>
      <definition>
        <name>TEST</name>
        <address-family>
          <ipv4/>
        </address-family>
      </definition>
    </vrf>
  </native>
</config>
'''

try:
    with m.locked(target='running'):
        edit_reply = m.edit_config(edit_data, target='running', format='xml')
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

#### Try to delete the VRF

In [ ]:
from ncclient.operations import TimeoutExpiredError

edit_data = '''
<config xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
    <vrf>
      <definition nc:operation="delete">
        <name>TEST</name>
      </definition>
    </vrf>
  </native>
</config>
'''

try:
    with m.locked(target='running'):
        edit_reply = m.edit_config(edit_data, target='running', format='xml')
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

#### Delete the VRF right

In [ ]:
from ncclient.operations import TimeoutExpiredError
import textwrap

# get all interfaces where VRF "TEST" is bound
xpath = "/native/interface/*/vrf[forwarding='TEST']"
c = m.get(filter=('xpath', xpath))

# set operation=delete on every VRF found
for e in c.data.xpath('//*[local-name()="vrf"]'):
    e.attrib['{urn:ietf:params:xml:ns:netconf:base:1.0}operation'] = "delete"

# find the node "native" and add it to a config node and then run an edit
config = etree.Element(
        "config",
        nsmap = {None: 'urn:ietf:params:xml:ns:netconf:base:1.0'})
config.append(c.data.xpath('//*[local-name()="native"]')[0])

# display what we will try to delete
print('What we will no try to delete:\n')
to_delete = etree.tostring(config, pretty_print=True).decode()
print(textwrap.indent(to_delete, '  ') )

# now delete the VRF from, the interface
try:
    with m.locked(target='running'):
        edit_reply = m.edit_config(config, target='running', format='xml')
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

#### Try put an interface into a non-existent VRF

In [ ]:
from ncclient.operations import TimeoutExpiredError

edit_data = '''
<config xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
    <interface>
      <GigabitEthernet>
        <name>3</name>
        <vrf>
          <forwarding>DOES_NOT_EXIST</forwarding>
        </vrf>
        <ip>
          <address>
            <primary>
              <address>192.168.2.222</address>
              <mask>255.255.255.0</mask>
            </primary>
          </address>
        </ip>
      </GigabitEthernet>
    </interface>
  </native>
</config>
'''

try:
    with m.locked(target='running'):
        edit_reply = m.edit_config(edit_data, target='running', format='xml')
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

### Alternate XML Creation

In [ ]:
from lxml import etree

def create_interface_filter(intf_name):
    interfaces = etree.Element(
        "interfaces",
        nsmap = {None: 'http://openconfig.net/yang/interfaces'})
    interface = etree.SubElement(interfaces, "interface")
    etree.SubElement(interface, 'name').text = intf_name
    state = etree.SubElement(interface, 'state')
    etree.SubElement(state, 'counters')
    return interfaces

print(etree.tostring(create_interface_filter('MgmtEth0/RP0/CPU0/0'), pretty_print=True))
print(etree.tostring(create_interface_filter('GigabitEthernet1'), pretty_print=True))

# Enable Debugging

In [ ]:
import logging

handler = logging.StreamHandler()
for l in ['ncclient.transport.ssh', 'ncclient.transport.session', 'ncclient.operations.rpc']:
    logger = logging.getLogger(l)
    logger.addHandler(handler)
    logger.setLevel(logging.DEBUG)